<a href="https://colab.research.google.com/github/yunhyeongil/pandas_base/blob/main/%EB%A1%9C%EC%9D%B4%ED%84%B0%EB%89%B4%EC%8A%A42_%EC%9C%A4%ED%98%95%EC%9D%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sequential 신경망

In [ ]:
from tensorflow.keras.datasets import reuters
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [ ]:
from tensorflow.keras.datasets import reuters
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

In [ ]:
(X_train, y_train), (X_test, y_test) = reuters.load_data(num_words=1000, test_split=0.2)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/reuters.py:143: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/reuters.py:144: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


In [ ]:
print('훈련용 뉴스 기사 : {}'.format(len(X_train)))
print('테스트용 뉴스 기사 : {}'.format(len(X_test)))
num_classes = max(y_train) + 1
print('카테고리 : {}'.format(num_classes))

훈련용 뉴스 기사 : 8982
테스트용 뉴스 기사 : 2246
카테고리 : 46


In [ ]:
print(X_train[0]) # 첫번째 훈련용 뉴스 기사
print(y_train[0]) # 첫번째 훈련용 뉴스 기사의 레이블
print(len(X_train[0]))

[1, 2, 2, 8, 43, 10, 447, 5, 25, 207, 270, 5, 2, 111, 16, 369, 186, 90, 67, 7, 89, 5, 19, 102, 6, 19, 124, 15, 90, 67, 84, 22, 482, 26, 7, 48, 4, 49, 8, 864, 39, 209, 154, 6, 151, 6, 83, 11, 15, 22, 155, 11, 15, 7, 48, 9, 2, 2, 504, 6, 258, 6, 272, 11, 15, 22, 134, 44, 11, 15, 16, 8, 197, 2, 90, 67, 52, 29, 209, 30, 32, 132, 6, 109, 15, 17, 12]
3
87


In [ ]:
max_len = max(len(l) for l in X_train)
max_len = 100

In [ ]:
max_len = 100
X_train = pad_sequences(X_train, maxlen=max_len) # 훈련용 뉴스 기사 패딩
X_test = pad_sequences(X_test, maxlen=max_len) # 테스트용 뉴스 기사 패딩

In [ ]:
print(X_train[0])
print(len(X_train))
print(len(X_train[0]))

[  0   0   0   0   0   0   0   0   0   0   0   0   0   1   2   2   8  43
  10 447   5  25 207 270   5   2 111  16 369 186  90  67   7  89   5  19
 102   6  19 124  15  90  67  84  22 482  26   7  48   4  49   8 864  39
 209 154   6 151   6  83  11  15  22 155  11  15   7  48   9   2   2 504
   6 258   6 272  11  15  22 134  44  11  15  16   8 197   2  90  67  52
  29 209  30  32 132   6 109  15  17  12]
8982
100


In [ ]:
print(y_train)
print(len(y_train[0]))
print(len(y_train))

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
46
8982


In [ ]:
y_train = to_categorical(y_train) # 훈련용 뉴스 기사 레이블의 원-핫 인코딩
y_test = to_categorical(y_test) # 테스트용 뉴스 기사 레이블의 원-핫 인코딩

In [ ]:
print(y_train[0])

[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [ ]:
word_index = reuters.get_word_index(path="reuters_word_index.json")

557056/550378 [==============================] - 0s 0us/step


In [ ]:
print(len(word_index.keys()))

30979


In [ ]:
index_to_word = {index +3 : word for word, index in word_index.items()}

In [ ]:
for index, token in enumerate(("<pad>", "<sos>", "<unk>")):
  index_to_word[index] = token

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten

model = Sequential()
model.add(Embedding(len(word_index.keys()),200, input_length=max_len))
model.add(Flatten())
model.add(Dense(46, activation="softmax"))

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 100, 4)            123916    
_________________________________________________________________
flatten_2 (Flatten)          (None, 400)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 46)                18446     
Total params: 142,362
Trainable params: 142,362
Non-trainable params: 0
_________________________________________________________________


In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4) # 훈련 조기 종료
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True) # 훈련 과정에서 검증 데이터가 가장 높았을 때 저장

model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train,  batch_size=128, epochs=30, callbacks=[es, mc], validation_data=(X_test, y_test))

Epoch 1/30
71/71 [==============================] - 7s 96ms/step - loss: 0.2165 - acc: 0.9454 - val_loss: 1.4629 - val_acc: 0.6812

Epoch 00001: val_acc improved from -inf to 0.68121, saving model to best_model.h5
Epoch 2/30
71/71 [==============================] - 7s 94ms/step - loss: 0.1482 - acc: 0.9535 - val_loss: 1.5080 - val_acc: 0.6808

Epoch 00002: val_acc did not improve from 0.68121
Epoch 3/30
71/71 [==============================] - 7s 94ms/step - loss: 0.1414 - acc: 0.9530 - val_loss: 1.5680 - val_acc: 0.6772

Epoch 00003: val_acc did not improve from 0.68121
Epoch 4/30
71/71 [==============================] - 7s 93ms/step - loss: 0.1348 - acc: 0.9555 - val_loss: 1.5399 - val_acc: 0.6870

Epoch 00004: val_acc improved from 0.68121 to 0.68700, saving model to best_model.h5
Epoch 5/30
71/71 [==============================] - 7s 94ms/step - loss: 0.1382 - acc: 0.9521 - val_loss: 1.5893 - val_acc: 0.6808

Epoch 00005: val_acc did not improve from 0.68700
Epoch 00005: early stop

In [ ]:
loaded_model = load_model('best_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

71/71 [==============================] - 0s 3ms/step - loss: 1.5399 - acc: 0.6870

 테스트 정확도: 0.6870


# 결과 0.6870

# RNN의 일종인  LSTM

In [ ]:
model = Sequential()
model.add(Embedding(1000, 120))
model.add(LSTM(120))
model.add(Dense(46, activation='softmax'))

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [ ]:
history = model.fit(X_train, y_train, batch_size=128, epochs=30, callbacks=[es, mc], validation_data=(X_test, y_test))

Epoch 1/30
71/71 [==============================] - 26s 345ms/step - loss: 2.6356 - acc: 0.3387 - val_loss: 2.4019 - val_acc: 0.3620

Epoch 00001: val_acc improved from -inf to 0.36198, saving model to best_model.h5
Epoch 2/30
71/71 [==============================] - 24s 339ms/step - loss: 2.1429 - acc: 0.4371 - val_loss: 1.9664 - val_acc: 0.5022

Epoch 00002: val_acc improved from 0.36198 to 0.50223, saving model to best_model.h5
Epoch 3/30
71/71 [==============================] - 24s 338ms/step - loss: 1.8895 - acc: 0.5219 - val_loss: 1.8051 - val_acc: 0.5530

Epoch 00003: val_acc improved from 0.50223 to 0.55298, saving model to best_model.h5
Epoch 4/30
71/71 [==============================] - 24s 338ms/step - loss: 1.7474 - acc: 0.5481 - val_loss: 1.7157 - val_acc: 0.5481

Epoch 00004: val_acc did not improve from 0.55298
Epoch 5/30
71/71 [==============================] - 24s 344ms/step - loss: 1.6815 - acc: 0.5662 - val_loss: 1.6878 - val_acc: 0.5721

Epoch 00005: val_acc improve

In [ ]:
ded_model = load_model('best_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

71/71 [==============================] - 0s 3ms/step - loss: 1.5399 - acc: 0.6870

 테스트 정확도: 0.6870


# 결과 :  0.6870

# 양방향 LSTM


In [ ]:
from tensorflow.keras.datasets import reuters
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [ ]:
max_words = 1000
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=max_words, test_split=0.2)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/reuters.py:143: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/reuters.py:144: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


In [ ]:
print(f'훈련 샘플의 수 : {len(x_train)}')
print(f'테스트 샘플의 수 : {len(x_test)}')
num_classes = max(y_train) + 1
print(f'클래스의 수 : {num_classes}')

훈련 샘플의 수 : 8982
테스트 샘플의 수 : 2246
클래스의 수 : 46


pad_sequences()<br>
keras.preprocessing.sequence.pad_sequences<br>(sequences, maxlen=None, dtype='int32', padding='pre', truncating='pre', value=0.0)

In [ ]:
max_len = 100
x_train = pad_sequences(x_train, maxlen=max_len) # 훈련용 뉴스 기사 패딩
x_test = pad_sequences(x_test, maxlen=max_len) # 테스트용 뉴스 기사 패딩

In [ ]:
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train shape: (8982, 100)
x_test shape: (2246, 100)


In [ ]:
y_train = to_categorical(y_train, num_classes) # 훈련용 뉴스 기사 레이블의 원-핫 인코딩
y_test = to_categorical(y_test, num_classes) # 테스트용 뉴스 기사 레이블의 원-핫 인코딩

In [ ]:
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

y_train shape: (8982, 46)
y_test shape: (2246, 46)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout, Activation, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

In [ ]:
model = Sequential()
model.add(Embedding(max_words, 128, input_length=max_len))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4) # 훈련 조기 종료
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True) # 훈련 과정에서 검증 데이터가 가장 높았을 때 저장

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [ ]:
history = model.fit(x_train, y_train, batch_size=128, epochs=30, callbacks=[es, mc], validation_data=(x_test, y_test))

Epoch 1/30
71/71 [==============================] - 28s 352ms/step - loss: 2.6372 - acc: 0.3378 - val_loss: 2.1319 - val_acc: 0.3744

Epoch 00001: val_acc improved from -inf to 0.37444, saving model to best_model.h5
Epoch 2/30
71/71 [==============================] - 24s 345ms/step - loss: 1.9918 - acc: 0.4738 - val_loss: 1.8340 - val_acc: 0.5062

Epoch 00002: val_acc improved from 0.37444 to 0.50623, saving model to best_model.h5
Epoch 3/30
71/71 [==============================] - 25s 345ms/step - loss: 1.7524 - acc: 0.5411 - val_loss: 1.6752 - val_acc: 0.5708

Epoch 00003: val_acc improved from 0.50623 to 0.57079, saving model to best_model.h5
Epoch 4/30
71/71 [==============================] - 24s 345ms/step - loss: 1.6868 - acc: 0.5599 - val_loss: 1.6544 - val_acc: 0.5699

Epoch 00004: val_acc did not improve from 0.57079
Epoch 5/30
71/71 [==============================] - 24s 341ms/step - loss: 1.5624 - acc: 0.5943 - val_loss: 1.5479 - val_acc: 0.6020

Epoch 00005: val_acc improve

In [ ]:
loaded_model = load_model('best_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(x_test, y_test)[1]))

71/71 [==============================] - 3s 27ms/step - loss: 1.2849 - acc: 0.7012

 테스트 정확도: 0.7012


# 결과 :  0.7012

# TF-IDF-ADM

In [ ]:
from tensorflow.keras.datasets import reuters

In [ ]:
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=None, test_split=0.2)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/reuters.py:143: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/reuters.py:144: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


In [ ]:
word_index = reuters.get_word_index(path="reuters_word_index.json")

In [ ]:
print(f'훈련 샘플의 수 : {len(x_train)}') # train sequences
print(f'테스트 샘플의 수 : {len(x_test)}') # test sequences
num_classes = max(y_train) + 1
print(f'클래스의 수 : {num_classes}')

훈련 샘플의 수 : 8982
테스트 샘플의 수 : 2246
클래스의 수 : 46


In [ ]:
index_to_word = {}
for key, value in word_index.items():
    index_to_word[value] = key

In [ ]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

print(x_train[:5])

(8982,)
(2246,)
(8982,)
(2246,)
[list([1, 27595, 28842, 8, 43, 10, 447, 5, 25, 207, 270, 5, 3095, 111, 16, 369, 186, 90, 67, 7, 89, 5, 19, 102, 6, 19, 124, 15, 90, 67, 84, 22, 482, 26, 7, 48, 4, 49, 8, 864, 39, 209, 154, 6, 151, 6, 83, 11, 15, 22, 155, 11, 15, 7, 48, 9, 4579, 1005, 504, 6, 258, 6, 272, 11, 15, 22, 134, 44, 11, 15, 16, 8, 197, 1245, 90, 67, 52, 29, 209, 30, 32, 132, 6, 109, 15, 17, 12])
 list([1, 3267, 699, 3434, 2295, 56, 16784, 7511, 9, 56, 3906, 1073, 81, 5, 1198, 57, 366, 737, 132, 20, 4093, 7, 19261, 49, 2295, 13415, 1037, 3267, 699, 3434, 8, 7, 10, 241, 16, 855, 129, 231, 783, 5, 4, 587, 2295, 13415, 30625, 775, 7, 48, 34, 191, 44, 35, 1795, 505, 17, 12])
 list([1, 53, 12, 284, 15, 14, 272, 26, 53, 959, 32, 818, 15, 14, 272, 26, 39, 684, 70, 11, 14, 12, 3886, 18, 180, 183, 187, 70, 11, 14, 102, 32, 11, 29, 53, 44, 704, 15, 14, 19, 758, 15, 53, 959, 47, 1013, 15, 14, 19, 132, 15, 39, 965, 32, 11, 14, 147, 72, 11, 180, 183, 187, 44, 11, 14, 102, 19, 11, 123, 186, 90

In [ ]:
print(' '.join([index_to_word[x] for x in x_train[0]]))
print(y_train[0])

the wattie nondiscriminatory mln loss for plc said at only ended said commonwealth could 1 traders now april 0 a after said from 1985 and from foreign 000 april 0 prices its account year a but in this mln home an states earlier and rise and revs vs 000 its 16 vs 000 a but 3 psbr oils several and shareholders and dividend vs 000 its all 4 vs 000 1 mln agreed largely april 0 are 2 states will billion total and against 000 pct dlrs
3


In [ ]:
import keras
import numpy as np
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer

from tensorflow.keras.preprocessing.text import Tokenizer

test_seq = [[1,2,3,4,5,6]]

tok = Tokenizer(num_words=10)
tok.fit_on_sequences(test_seq)

tok.sequences_to_matrix(test_seq)
array([[0., 1., 1., 1., 1., 1., 1., 0., 0., 0.]])

In [ ]:
max_words= 10000
tokenizer= Tokenizer(num_words=max_words)
print(x_train.tolist()[:5])
print(x_test.tolist()[:5])
tokenizer.fit_on_sequences(x_train.tolist())
tokenizer.fit_on_sequences(x_test.tolist())

print(tokenizer.sequences_to_matrix(x_train.tolist()))

x_train = np.asarray(tokenizer.sequences_to_matrix(x_train.tolist(), mode="tfidf"))
x_test = np.asarray(tokenizer.sequences_to_matrix(x_test.tolist(), mode="tfidf"))

[[1, 27595, 28842, 8, 43, 10, 447, 5, 25, 207, 270, 5, 3095, 111, 16, 369, 186, 90, 67, 7, 89, 5, 19, 102, 6, 19, 124, 15, 90, 67, 84, 22, 482, 26, 7, 48, 4, 49, 8, 864, 39, 209, 154, 6, 151, 6, 83, 11, 15, 22, 155, 11, 15, 7, 48, 9, 4579, 1005, 504, 6, 258, 6, 272, 11, 15, 22, 134, 44, 11, 15, 16, 8, 197, 1245, 90, 67, 52, 29, 209, 30, 32, 132, 6, 109, 15, 17, 12], [1, 3267, 699, 3434, 2295, 56, 16784, 7511, 9, 56, 3906, 1073, 81, 5, 1198, 57, 366, 737, 132, 20, 4093, 7, 19261, 49, 2295, 13415, 1037, 3267, 699, 3434, 8, 7, 10, 241, 16, 855, 129, 231, 783, 5, 4, 587, 2295, 13415, 30625, 775, 7, 48, 34, 191, 44, 35, 1795, 505, 17, 12], [1, 53, 12, 284, 15, 14, 272, 26, 53, 959, 32, 818, 15, 14, 272, 26, 39, 684, 70, 11, 14, 12, 3886, 18, 180, 183, 187, 70, 11, 14, 102, 32, 11, 29, 53, 44, 704, 15, 14, 19, 758, 15, 53, 959, 47, 1013, 15, 14, 19, 132, 15, 39, 965, 32, 11, 14, 147, 72, 11, 180, 183, 187, 44, 11, 14, 102, 19, 11, 123, 186, 90, 67, 960, 4, 78, 13, 68, 467, 511, 110, 59, 89, 

In [ ]:
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train shape: (8982, 10000)
x_test shape: (2246, 10000)


In [ ]:
print(x_train[:5])
print(x_test[:5])

[[0.         0.69310265 0.         ... 0.         0.         0.        ]
 [0.         0.69310265 0.         ... 0.         0.         0.        ]
 [0.         0.69310265 0.         ... 0.         0.         0.        ]
 [0.         0.69310265 0.         ... 0.         0.         0.        ]
 [0.         0.69310265 0.         ... 0.         0.         0.        ]]
[[0.         0.69310265 0.         ... 0.         0.         0.        ]
 [0.         0.69310265 0.         ... 0.         0.         0.        ]
 [0.         0.69310265 0.         ... 0.         0.         0.        ]
 [0.         0.69310265 0.         ... 0.         0.         0.        ]
 [0.         0.69310265 0.         ... 0.         0.         0.        ]]


In [ ]:
y_train = to_categorical(y_train, num_classes) # 훈련용 뉴스 기사 레이블의 원-핫 인코딩
y_test = to_categorical(y_test, num_classes) # 테스트용 뉴스 기사 레이블의 원-핫 인코딩

In [ ]:
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

y_train shape: (8982, 46)
y_test shape: (2246, 46)


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

In [ ]:
# 모델은 인터넷에서 찾아본것 중 테스트 정확도가 높게 나온걸 사용했다..
model = Sequential()
model.add(Dense(512, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4) # 훈련 조기 종료
mc = ModelCheckpoint('t_best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True) # 훈련 과정에서 검증 데이터가 가장 높았을 때 저장

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [ ]:
history = model.fit(x_train, y_train, batch_size=128, epochs=30, callbacks=[es, mc], validation_data=(x_test, y_test))

Epoch 1/30
71/71 [==============================] - 6s 73ms/step - loss: 1.8900 - acc: 0.6129 - val_loss: 0.9047 - val_acc: 0.8121

Epoch 00001: val_acc improved from -inf to 0.81211, saving model to t_best_model.h5
Epoch 2/30
71/71 [==============================] - 5s 69ms/step - loss: 0.3896 - acc: 0.9200 - val_loss: 0.9296 - val_acc: 0.8143

Epoch 00002: val_acc improved from 0.81211 to 0.81434, saving model to t_best_model.h5
Epoch 3/30
71/71 [==============================] - 5s 68ms/step - loss: 0.2335 - acc: 0.9484 - val_loss: 0.9907 - val_acc: 0.8081

Epoch 00003: val_acc did not improve from 0.81434
Epoch 4/30
71/71 [==============================] - 5s 67ms/step - loss: 0.1981 - acc: 0.9539 - val_loss: 1.0522 - val_acc: 0.8112

Epoch 00004: val_acc did not improve from 0.81434
Epoch 5/30
71/71 [==============================] - 5s 68ms/step - loss: 0.1823 - acc: 0.9585 - val_loss: 1.0929 - val_acc: 0.8112

Epoch 00005: val_acc did not improve from 0.81434
Epoch 00005: early 

In [ ]:
loaded_model = load_model('t_best_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(x_test, y_test)[1]))

71/71 [==============================] - 1s 19ms/step - loss: 0.9296 - acc: 0.8143

 테스트 정확도: 0.8143


#결과 : 0.8143

# TF-IDF-ADAMAX

In [ ]:
from tensorflow.keras.datasets import reuters

In [ ]:
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=None, test_split=0.2)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/reuters.py:143: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/reuters.py:144: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


In [ ]:
word_index = reuters.get_word_index(path="reuters_word_index.json")

In [ ]:
print(f'훈련 샘플의 수 : {len(x_train)}') # train sequences
print(f'테스트 샘플의 수 : {len(x_test)}') # test sequences
num_classes = max(y_train) + 1
print(f'클래스의 수 : {num_classes}')

훈련 샘플의 수 : 8982
테스트 샘플의 수 : 2246
클래스의 수 : 46


In [ ]:
index_to_word = {}
for key, value in word_index.items():
    index_to_word[value] = key

In [ ]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(8982,)
(2246,)
(8982,)
(2246,)


In [ ]:
print(' '.join([index_to_word[x] for x in x_train[0]]))
print(y_train[0])

the wattie nondiscriminatory mln loss for plc said at only ended said commonwealth could 1 traders now april 0 a after said from 1985 and from foreign 000 april 0 prices its account year a but in this mln home an states earlier and rise and revs vs 000 its 16 vs 000 a but 3 psbr oils several and shareholders and dividend vs 000 its all 4 vs 000 1 mln agreed largely april 0 are 2 states will billion total and against 000 pct dlrs
3


In [ ]:
import keras
import numpy as np
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer

In [ ]:
# Vectorizing sequence data
max_words= 10000
tokenizer= Tokenizer(num_words=max_words)
tokenizer.fit_on_sequences(x_train.tolist())
tokenizer.fit_on_sequences(x_test.tolist())
x_train = np.asarray(tokenizer.sequences_to_matrix(x_train.tolist(), mode="tfidf"))
x_test = np.asarray(tokenizer.sequences_to_matrix(x_test.tolist(), mode="tfidf"))

In [ ]:
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train shape: (8982, 10000)
x_test shape: (2246, 10000)


In [ ]:
y_train = to_categorical(y_train, num_classes) # 훈련용 뉴스 기사 레이블의 원-핫 인코딩
y_test = to_categorical(y_test, num_classes) # 테스트용 뉴스 기사 레이블의 원-핫 인코딩

In [ ]:
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

y_train shape: (8982, 46)
y_test shape: (2246, 46)


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

In [ ]:
model = Sequential()
model.add(Dense(512, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4) # 훈련 조기 종료
mc = ModelCheckpoint('./t_best_model_adamax.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True) # 훈련 과정에서 검증 데이터가 가장 높았을 때 저장

In [ ]:

model.compile(loss='categorical_crossentropy', optimizer='adamax', metrics=['acc']) # Adamax # 무한 노름(infinity norm)에 기반한 Adam의 변형

In [ ]:
history = model.fit(x_train, y_train, batch_size=128, epochs=100, callbacks=[es, mc], validation_data=(x_test, y_test))

Epoch 1/100
71/71 [==============================] - 6s 73ms/step - loss: 2.1092 - acc: 0.5723 - val_loss: 1.1281 - val_acc: 0.7614

Epoch 00001: val_acc improved from -inf to 0.76135, saving model to ./t_best_model_adamax.h5
Epoch 2/100
71/71 [==============================] - 5s 68ms/step - loss: 0.7998 - acc: 0.8342 - val_loss: 0.9495 - val_acc: 0.7965

Epoch 00002: val_acc improved from 0.76135 to 0.79653, saving model to ./t_best_model_adamax.h5
Epoch 3/100
71/71 [==============================] - 5s 68ms/step - loss: 0.5261 - acc: 0.8934 - val_loss: 0.8836 - val_acc: 0.8081

Epoch 00003: val_acc improved from 0.79653 to 0.80810, saving model to ./t_best_model_adamax.h5
Epoch 4/100
71/71 [==============================] - 5s 69ms/step - loss: 0.3931 - acc: 0.9196 - val_loss: 0.8647 - val_acc: 0.8108

Epoch 00004: val_acc improved from 0.80810 to 0.81077, saving model to ./t_best_model_adamax.h5
Epoch 5/100
71/71 [==============================] - 5s 68ms/step - loss: 0.2952 - acc:

In [ ]:
loaded_model = load_model('t_best_model_adamax.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(x_test, y_test)[1]))

71/71 [==============================] - 1s 10ms/step - loss: 0.9005 - acc: 0.8152

 테스트 정확도: 0.8152


# 결과: 0.8152

#TF-IDF- SGD 확률적 경사하강법

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4) # 훈련 조기 종료
mc = ModelCheckpoint('./t_best_model_sgd.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True) # 훈련 과정에서 검증 데이터가 가장 높았을 때 저장

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['acc'])  # 확률적 경사 하강법 


In [ ]:
history = model.fit(x_train, y_train, batch_size=128, epochs=100, callbacks=[es, mc], validation_data=(x_test, y_test))

Epoch 1/100
71/71 [==============================] - 7s 92ms/step - loss: 0.1224 - acc: 0.9668 - val_loss: 1.6833 - val_acc: 0.8028

Epoch 00001: val_acc improved from -inf to 0.80276, saving model to ./t_best_model_sgd.h5
Epoch 2/100
71/71 [==============================] - 5s 71ms/step - loss: 0.1433 - acc: 0.9638 - val_loss: 1.6633 - val_acc: 0.8032

Epoch 00002: val_acc improved from 0.80276 to 0.80321, saving model to ./t_best_model_sgd.h5
Epoch 3/100
71/71 [==============================] - 7s 92ms/step - loss: 0.1596 - acc: 0.9612 - val_loss: 1.6465 - val_acc: 0.8032

Epoch 00003: val_acc did not improve from 0.80321
Epoch 4/100
71/71 [==============================] - 4s 60ms/step - loss: 0.1354 - acc: 0.9626 - val_loss: 1.6348 - val_acc: 0.8041

Epoch 00004: val_acc improved from 0.80321 to 0.80410, saving model to ./t_best_model_sgd.h5
Epoch 5/100
71/71 [==============================] - 4s 60ms/step - loss: 0.1267 - acc: 0.9639 - val_loss: 1.6297 - val_acc: 0.8050

Epoch 000

In [ ]:
loaded_model = load_model('t_best_model_sgd.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(x_test, y_test)[1]))

71/71 [==============================] - 1s 10ms/step - loss: 1.5984 - acc: 0.8068

 테스트 정확도: 0.8068


# 결과 : 0.8068

#TF-IDF RMS prop

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4) # 훈련 조기 종료
mc = ModelCheckpoint('./t_best_model_RMS.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True) # 훈련 과정에서 검증 데이터가 가장 높았을 때 저장

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['acc']) # RMSProp # 일반적으로 순환 신경망(Recurrent Neural Networks)의 옵티마이저로 많이 사용

In [ ]:
history = model.fit(x_train, y_train, batch_size=128, epochs=100, callbacks=[es, mc], validation_data=(x_test, y_test))

Epoch 1/100
71/71 [==============================] - 7s 93ms/step - loss: 0.1734 - acc: 0.9575 - val_loss: 1.7994 - val_acc: 0.8054

Epoch 00001: val_acc improved from -inf to 0.80543, saving model to ./t_best_model_RMS.h5
Epoch 2/100
71/71 [==============================] - 6s 88ms/step - loss: 0.1791 - acc: 0.9558 - val_loss: 1.9421 - val_acc: 0.8023

Epoch 00002: val_acc did not improve from 0.80543
Epoch 3/100
71/71 [==============================] - 6s 89ms/step - loss: 0.1725 - acc: 0.9583 - val_loss: 2.0778 - val_acc: 0.7983

Epoch 00003: val_acc did not improve from 0.80543
Epoch 4/100
71/71 [==============================] - 6s 88ms/step - loss: 0.1632 - acc: 0.9619 - val_loss: 2.2791 - val_acc: 0.7965

Epoch 00004: val_acc did not improve from 0.80543
Epoch 5/100
71/71 [==============================] - 6s 89ms/step - loss: 0.1691 - acc: 0.9569 - val_loss: 2.4562 - val_acc: 0.7934

Epoch 00005: val_acc did not improve from 0.80543
Epoch 00005: early stopping


In [ ]:
loaded_model = load_model('t_best_model_RMS.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(x_test, y_test)[1]))

71/71 [==============================] - 1s 10ms/step - loss: 1.7994 - acc: 0.8054

 테스트 정확도: 0.8054


# 결과 : 0.8054

# Word2Vector

In [ ]:
from tensorflow.keras.datasets import reuters

In [ ]:
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=None, test_split=0.2)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/reuters.py:143: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/reuters.py:144: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


In [ ]:
word_index = reuters.get_word_index(path="reuters_word_index.json")

In [ ]:
print(f'훈련 샘플의 수 : {len(x_train)}') # train sequences
print(f'테스트 샘플의 수 : {len(x_test)}') # test sequences
num_classes = max(y_train) + 1
print(f'클래스의 수 : {num_classes}')

훈련 샘플의 수 : 8982
테스트 샘플의 수 : 2246
클래스의 수 : 46


In [ ]:
index_to_word = {}
for key, value in word_index.items():
    index_to_word[value] = key

In [ ]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)
print(x_train[:5])

(8982,)
(2246,)
(8982,)
(2246,)
[list([1, 27595, 28842, 8, 43, 10, 447, 5, 25, 207, 270, 5, 3095, 111, 16, 369, 186, 90, 67, 7, 89, 5, 19, 102, 6, 19, 124, 15, 90, 67, 84, 22, 482, 26, 7, 48, 4, 49, 8, 864, 39, 209, 154, 6, 151, 6, 83, 11, 15, 22, 155, 11, 15, 7, 48, 9, 4579, 1005, 504, 6, 258, 6, 272, 11, 15, 22, 134, 44, 11, 15, 16, 8, 197, 1245, 90, 67, 52, 29, 209, 30, 32, 132, 6, 109, 15, 17, 12])
 list([1, 3267, 699, 3434, 2295, 56, 16784, 7511, 9, 56, 3906, 1073, 81, 5, 1198, 57, 366, 737, 132, 20, 4093, 7, 19261, 49, 2295, 13415, 1037, 3267, 699, 3434, 8, 7, 10, 241, 16, 855, 129, 231, 783, 5, 4, 587, 2295, 13415, 30625, 775, 7, 48, 34, 191, 44, 35, 1795, 505, 17, 12])
 list([1, 53, 12, 284, 15, 14, 272, 26, 53, 959, 32, 818, 15, 14, 272, 26, 39, 684, 70, 11, 14, 12, 3886, 18, 180, 183, 187, 70, 11, 14, 102, 32, 11, 29, 53, 44, 704, 15, 14, 19, 758, 15, 53, 959, 47, 1013, 15, 14, 19, 132, 15, 39, 965, 32, 11, 14, 147, 72, 11, 180, 183, 187, 44, 11, 14, 102, 19, 11, 123, 186, 90

In [2]:
import keras
import numpy as np
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.datasets import reuters
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

In [3]:
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=10000, test_split=0.2)

print(x_train)
print(y_train)

2113536/2110848 [==============================] - 0s 0us/step
[list([1, 2, 2, 8, 43, 10, 447, 5, 25, 207, 270, 5, 3095, 111, 16, 369, 186, 90, 67, 7, 89, 5, 19, 102, 6, 19, 124, 15, 90, 67, 84, 22, 482, 26, 7, 48, 4, 49, 8, 864, 39, 209, 154, 6, 151, 6, 83, 11, 15, 22, 155, 11, 15, 7, 48, 9, 4579, 1005, 504, 6, 258, 6, 272, 11, 15, 22, 134, 44, 11, 15, 16, 8, 197, 1245, 90, 67, 52, 29, 209, 30, 32, 132, 6, 109, 15, 17, 12])
 list([1, 3267, 699, 3434, 2295, 56, 2, 7511, 9, 56, 3906, 1073, 81, 5, 1198, 57, 366, 737, 132, 20, 4093, 7, 2, 49, 2295, 2, 1037, 3267, 699, 3434, 8, 7, 10, 241, 16, 855, 129, 231, 783, 5, 4, 587, 2295, 2, 2, 775, 7, 48, 34, 191, 44, 35, 1795, 505, 17, 12])
 list([1, 53, 12, 284, 15, 14, 272, 26, 53, 959, 32, 818, 15, 14, 272, 26, 39, 684, 70, 11, 14, 12, 3886, 18, 180, 183, 187, 70, 11, 14, 102, 32, 11, 29, 53, 44, 704, 15, 14, 19, 758, 15, 53, 959, 47, 1013, 15, 14, 19, 132, 15, 39, 965, 32, 11, 14, 147, 72, 11, 180, 183, 187, 44, 11, 14, 102, 19, 11, 123, 186,

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/reuters.py:143: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/reuters.py:144: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


In [4]:
# 분류된 기사 종류의 수가 46개
num_classes = max(y_train) + 1
print('클래스의 수 : {}'.format(num_classes))

클래스의 수 : 46


In [5]:
word_index = reuters.get_word_index(path="reuters_word_index.json")

557056/550378 [==============================] - 0s 0us/step


In [6]:
index_to_word = {index +3 : word for word, index in word_index.items()}

In [7]:
# 로이터의 데이타 특성을 위해 3가지 종류를 index_to_word에 추가
# 0 <pad>
# 1 <sos>
# 2 <unk>
for index, token in enumerate(("<pad>", "<sos>", "<unk>")):
  index_to_word[index] = token

print(x_train[0])
print(index_to_word[0])

[1, 2, 2, 8, 43, 10, 447, 5, 25, 207, 270, 5, 3095, 111, 16, 369, 186, 90, 67, 7, 89, 5, 19, 102, 6, 19, 124, 15, 90, 67, 84, 22, 482, 26, 7, 48, 4, 49, 8, 864, 39, 209, 154, 6, 151, 6, 83, 11, 15, 22, 155, 11, 15, 7, 48, 9, 4579, 1005, 504, 6, 258, 6, 272, 11, 15, 22, 134, 44, 11, 15, 16, 8, 197, 1245, 90, 67, 52, 29, 209, 30, 32, 132, 6, 109, 15, 17, 12]
<pad>


In [8]:
print(' '.join([index_to_word[index] for index in x_train[0]]))

<sos> <unk> <unk> said as a result of its december acquisition of space co it expects earnings per share in 1987 of 1 15 to 1 30 dlrs per share up from 70 cts in 1986 the company said pretax net should rise to nine to 10 mln dlrs from six mln dlrs in 1986 and rental operation revenues to 19 to 22 mln dlrs from 12 5 mln dlrs it said cash flow per share this year should be 2 50 to three dlrs reuter 3


In [9]:
decoded = []
for i in range(len(x_train)):
  t = ' '.join([index_to_word[index] for index in x_train[i]])
  decoded.append(t)

x_train = decoded

In [10]:
decoded = []
for i in range(len(x_test)):
  t = ' '.join([index_to_word[index] for index in x_test[i]])
  decoded.append(t)

x_test = decoded

In [25]:
import nltk
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize, sent_tokenize
nltk.download('abc')
nltk.download('punkt')

tokenized_data = []
for sentence in x_train:
    temp_X = word_tokenize(sentence) # 토큰화
    temp_X = [word for word in temp_X ]   
    tokenized_data.append(temp_X)

tokenized_data2 = []
for sentence in x_test:
    temp_X = word_tokenize(sentence) # 토큰화
    temp_X = [word for word in temp_X ]   
    tokenized_data2.append(temp_X)

[nltk_data] Downloading package abc to /root/nltk_data...
[nltk_data]   Package abc is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [26]:
from gensim.models import Word2Vec
model1 = Word2Vec(sentences = tokenized_data, window = 5, min_count = 5, workers = 4, sg = 0)
model2 = Word2Vec(sentences = tokenized_data2, window = 5, min_count = 5, workers = 4, sg = 0)

print(model1.vector_size)
print(model2.vector_size)

100
100


In [16]:
print(len(tokenized_data))
print(tokenized_data[:10])

8982
[['<', 'sos', '>', '<', 'unk', '>', '<', 'unk', '>', 'said', 'as', 'a', 'result', 'of', 'its', 'december', 'acquisition', 'of', 'space', 'co', 'it', 'expects', 'earnings', 'per', 'share', 'in', '1987', 'of', '1', '15', 'to', '1', '30', 'dlrs', 'per', 'share', 'up', 'from', '70', 'cts', 'in', '1986', 'the', 'company', 'said', 'pretax', 'net', 'should', 'rise', 'to', 'nine', 'to', '10', 'mln', 'dlrs', 'from', 'six', 'mln', 'dlrs', 'in', '1986', 'and', 'rental', 'operation', 'revenues', 'to', '19', 'to', '22', 'mln', 'dlrs', 'from', '12', '5', 'mln', 'dlrs', 'it', 'said', 'cash', 'flow', 'per', 'share', 'this', 'year', 'should', 'be', '2', '50', 'to', 'three', 'dlrs', 'reuter', '3'], ['<', 'sos', '>', 'generale', 'de', 'banque', 'sa', 'lt', '<', 'unk', '>', 'br', 'and', 'lt', 'heller', 'overseas', 'corp', 'of', 'chicago', 'have', 'each', 'taken', '50', 'pct', 'stakes', 'in', '<', 'unk', '>', 'company', 'sa', '<', 'unk', '>', 'factors', 'generale', 'de', 'banque', 'said', 'in', 'a', '

In [27]:
model_result = model1.wv.most_similar("december")

In [29]:
print(model_result)
print(model.vector_size)


[('january', 0.8733484148979187), ('february', 0.862957239151001), ('october', 0.8612182140350342), ('september', 0.8532811999320984), ('november', 0.8507401943206787), ('1984', 0.8181977272033691), ('august', 0.8009870052337646), ('1983', 0.7845745086669922), ('march', 0.7429894208908081), ('1980', 0.7244561910629272)]
100


In [19]:
from gensim.models import KeyedVectors

model.wv.save_word2vec_format('./w2v')
loaded_model = KeyedVectors.load_word2vec_format("./w2v")
print("모델 load완료")

모델 load완료


In [30]:
loaded_model.most_similar('acquisition')

[('transaction', 0.871552586555481),
 ('merger', 0.835892915725708),
 ('sale', 0.8280469179153442),
 ('completion', 0.8188002109527588),
 ('purchase', 0.8090455532073975),
 ('offering', 0.8047310709953308),
 ('shareholder', 0.8026955127716064),
 ('buyout', 0.780207633972168),
 ('borg', 0.7714020609855652),
 ('partnership', 0.7631629705429077)]

In [31]:
model1.wv.vectors.shape

(9472, 100)

In [33]:
print(model1.wv.vectors[:5])
print(len(model1.wv.vectors))

[[ 8.65486383e-01  1.50054836e+00  4.33019223e-03 -3.88313055e-01
  -1.13649988e+00 -5.22642016e-01 -1.52314448e+00  2.34618759e+00
  -1.32605839e+00  1.08324277e+00  9.06893238e-02  7.51058102e-01
  -2.14118052e+00 -1.15842247e+00 -5.51051497e-02 -7.29674280e-01
   5.03455698e-01  1.03322875e+00 -1.18476498e+00  1.50206581e-01
   1.24821830e+00  1.48341954e-01 -2.15648866e+00 -4.45207536e-01
  -2.77271777e-01 -2.13540971e-01 -5.27371824e-01  3.65279794e-01
  -3.74446779e-01  1.87801552e+00 -6.71217978e-01 -3.50813419e-01
  -1.20861506e+00  1.20612524e-01  1.75407767e-01  5.40230572e-01
  -9.03678656e-01  1.07454371e+00  1.22946650e-01 -6.60030007e-01
  -1.20570433e+00  1.61830083e-01  1.48381278e-01  1.41275001e+00
  -1.46235251e+00 -1.57062066e+00 -6.19161069e-01 -1.13653624e+00
   6.08898878e-01  1.40419865e+00  1.96992442e-01 -7.12930411e-02
  -2.56196596e-02  8.27655867e-02 -1.13556123e+00 -9.97310698e-01
  -1.46648258e-01  1.37497509e+00 -3.62385176e-02 -4.50748056e-01
   9.01021